In [17]:
import sys
import time
import random
import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import networkx as nx

from copy import deepcopy
from typing import Union, List, Tuple, Dict
from pgmpy.readwrite import XMLBIFReader

from BayesNet import *
from BNReasoner import *
from utils import *

# Initialize
---

In [151]:
# BIFXML files
net = './testing/KR_use_case.bifxml'

In [152]:
# constructs a BN object
bn = BayesNet()

# Loads the BN from an BIFXML file
bn.load_from_bifxml(net)

# create reasoner
bnr = BNReasoner(bn)

### PERFORMANCE EVALUATION

In [198]:
data = []

In [207]:
evi = {
        'Obesity': True, 
        'High Blood Pressure': True,
        'Sleep Deprivation': False
}
qry = ['Blurry Vision']

start = time.time()
MAP = bnr.MAP(query=qry, evidence=evi, ordering_function=minfill_order)
end = time.time()

print(f"MAP done in {round(end - start, 3)} seconds!\n")

print(f"MAP:\n{MAP}\n")

data.append(('MAP', 11, 'minfill', qry, evi, end-start))
data

MAP done in 1.532 seconds!

MAP:
{'f_mult(f_mult(f_mult(f_mult(f_mult(Heart Failure-f_mult(f_mult(Blurry Vision-f_mult(High Sugar Intake-f_mult(f_mult(Stress-Obesity-Sleep Deprivation-Headache-Diabetes-Digestive Problems-High Blood Pressure)_sum(Stress))_sum(Obesity))_sum(High Sugar Intake))_sum(Sleep Deprivation))_sum(Headache))_sum(Diabetes))_sum(Digestive Problems))_sum(Heart Failure))_sum(High Blood Pressure))_max(Blurry Vision)': p                     3.6959e-06
inst_Blurry Vision          True
Name: 1, dtype: object}



[('MAP',
  11,
  'random',
  ['Blurry Vision'],
  {'Obesity': True, 'High Blood Pressure': True, 'Sleep Deprivation': False},
  1.771449327468872),
 ('MAP',
  11,
  'random',
  ['Blurry Vision'],
  {'Obesity': True, 'High Blood Pressure': True, 'Sleep Deprivation': False},
  1.844426155090332),
 ('MAP',
  11,
  'random',
  ['Blurry Vision'],
  {'Obesity': True, 'High Blood Pressure': True, 'Sleep Deprivation': False},
  2.331270456314087),
 ('MAP',
  11,
  'mindeg',
  ['Blurry Vision'],
  {'Obesity': True, 'High Blood Pressure': True, 'Sleep Deprivation': False},
  0.696791410446167),
 ('MAP',
  11,
  'mindeg',
  ['Blurry Vision'],
  {'Obesity': True, 'High Blood Pressure': True, 'Sleep Deprivation': False},
  0.771782636642456),
 ('MAP',
  11,
  'mindeg',
  ['Blurry Vision'],
  {'Obesity': True, 'High Blood Pressure': True, 'Sleep Deprivation': False},
  0.7027873992919922),
 ('MAP',
  11,
  'minfill',
  ['Blurry Vision'],
  {'Obesity': True, 'High Blood Pressure': True, 'Sleep Depriv

In [188]:
data.pop()

('MAP',
 11,
 'minfill',
 ['Blurry Vision'],
 {'Headache': True, 'Diabetes': False, 'Sleep Deprivation': True},
 1.4665625095367432)

In [208]:
df = pd.DataFrame(data, columns=["Method", "Vars", "Ordering", "Query", "Evidence", "Runtime"])
df

,Method,Vars,Ordering,Query,Evidence,Runtime
0,MAP,11,random,[Blurry Vision],"{'Obesity': True, 'High Blood Pressure': True,...",1.771449
1,MAP,11,random,[Blurry Vision],"{'Obesity': True, 'High Blood Pressure': True,...",1.844426
2,MAP,11,random,[Blurry Vision],"{'Obesity': True, 'High Blood Pressure': True,...",2.331270
3,MAP,11,mindeg,[Blurry Vision],"{'Obesity': True, 'High Blood Pressure': True,...",0.696791
4,MAP,11,mindeg,[Blurry Vision],"{'Obesity': True, 'High Blood Pressure': True,...",0.771783
5,MAP,11,mindeg,[Blurry Vision],"{'Obesity': True, 'High Blood Pressure': True,...",0.702787
6,MAP,11,minfill,[Blurry Vision],"{'Obesity': True, 'High Blood Pressure': True,...",1.567511
7,MAP,11,minfill,[Blurry Vision],"{'Obesity': True, 'High Blood Pressure': True,...",1.478578
8,MAP,11,minfill,[Blurry Vision],"{'Obesity': True, 'High Blood Pressure': True,...",1.531543


In [209]:
# df.to_csv("outputs/map_11_q3.csv", sep='\t', index=False)

# MPE

In [244]:
data = []

In [253]:
evi = {'High Blood Pressure': True}

start = time.time()
MPE = bnr.MPE(evidence=evi, ordering_function=minfill_order)
end = time.time()

print(f"MPE done in {round(end - start, 3)} seconds!\n")

print(f"MPE:\n{MPE}")

data.append(('MPE', 11, 'minfill', evi, end-start))
data

MPE done in 0.781 seconds!

MPE:
{'f_mult(f_mult(f_mult(f_mult(f_mult(f_mult(Heart Failure-f_mult(Blurry Vision)_max(Blurry Vision))_max(Heart Failure)-f_mult(Stress-High Blood Pressure-f_mult(Depression)_max(Depression)-f_mult(High Sugar Intake-Headache-f_mult(Obesity-Sleep Deprivation-Diabetes-Digestive Problems)_max(Obesity))_max(High Sugar Intake))_max(Stress))_max(Sleep Deprivation))_max(Headache))_max(Diabetes))_max(Digestive Problems))_max(High Blood Pressure)': p                           1.09907e-05
inst_Digestive Problems            True
inst_High Blood Pressure           True
Name: 1, dtype: object}


[('MPE', 11, 'random', {'High Blood Pressure': True}, 2.0763185024261475),
 ('MPE', 11, 'random', {'High Blood Pressure': True}, 2.567189931869507),
 ('MPE', 11, 'random', {'High Blood Pressure': True}, 3.2119603157043457),
 ('MPE', 11, 'mindeg', {'High Blood Pressure': True}, 0.783759355545044),
 ('MPE', 11, 'mindeg', {'High Blood Pressure': True}, 0.8307197093963623),
 ('MPE', 11, 'mindeg', {'High Blood Pressure': True}, 0.7647652626037598),
 ('MPE', 11, 'minfill', {'High Blood Pressure': True}, 0.7767601013183594),
 ('MPE', 11, 'minfill', {'High Blood Pressure': True}, 0.8677539825439453),
 ('MPE', 11, 'minfill', {'High Blood Pressure': True}, 0.7807807922363281)]

In [217]:
data.pop()

('MPE', 11, 'random', {'Blurry Vision': True}, 5.269336700439453)

In [254]:
df = pd.DataFrame(data, columns=["Method", "Vars", "Ordering", "Evidence", "Runtime"])
df

,Method,Vars,Ordering,Evidence,Runtime
0,MPE,11,random,{'High Blood Pressure': True},2.076319
1,MPE,11,random,{'High Blood Pressure': True},2.567190
2,MPE,11,random,{'High Blood Pressure': True},3.211960
3,MPE,11,mindeg,{'High Blood Pressure': True},0.783759
4,MPE,11,mindeg,{'High Blood Pressure': True},0.830720
5,MPE,11,mindeg,{'High Blood Pressure': True},0.764765
6,MPE,11,minfill,{'High Blood Pressure': True},0.776760
7,MPE,11,minfill,{'High Blood Pressure': True},0.867754
8,MPE,11,minfill,{'High Blood Pressure': True},0.780781


In [255]:
# df.to_csv("outputs/mpe_11_q3.csv", sep='\t', index=False)

# Usage
---

In [ ]:
# query inputs
# evidence = {'hear-bark': True, 'family-out': True}
# x = ['light-on']
# prio = ['dog-out']

### D-separation
Checks whether x and y are d-separated, given (possibly empty) evidence. 

* x and y must be lists
* evidence is (always) a dict
* output is boolean

In [ ]:
# dsep = bnr.d_separated(prio, x, evidence)
# print(f"Are {prio} and {x} d-separated by evidence '{evidence.keys()} = {evidence.values()}'?\n\n{dsep}")

### Ordering functions for determining elimination order
inputs should be network structure
* y = list of query variables
* z = priority variables which are ordered first __[optional]__

In [ ]:
# pi1 = mindeg_order(net=bn, query=prio)
# pi2 = minfill_order(net=bn, query=prio)
# print(f"Variable orderings mindeg:\t{pi1}")
# print(f"Variable orderings minfill:\t{pi2}")

### Marginal distributions
with optional ordering function (name of function is input)

In [ ]:
# without evidence:
# prior_marginal = bnr.compute_marginal(query=x, ordering_function=random_order)
# print(f"\n\nPrior Marginal for Query {x}\n{prior_marginal}")

# with evidence:
# posterior_marginal = bnr.compute_marginal(query=x, evidence=evidence, ordering_function=minfill_order)
# print(f"\nPosterior Marginal for Query {x} with Evidence {evidence}\n{posterior_marginal}")

### Most likely instantiations

In [ ]:
# MAP:
# MAP = bnr.MAP(query=x, evidence=evidence)
# print(f"MAP:\n{[MAP[key] for key in MAP]}\n")

# MPE:
# MPE = bnr.MPE(evidence=evidence, ordering_function=minfill_order)
# print(f"MPE:\n{[MPE[key] for key in MPE]}")